<a href="https://colab.research.google.com/github/sanglee/BIML2019/blob/master/RNN_Autocomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
  Autocompletion of the last character of words
  Given the first three letters of a four-letters word, learn to predict the last letter 
"""

import tensorflow as tf
import keras
import numpy as np


vocab = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
         'h', 'i', 'j', 'k', 'l', 'm', 'n',
         'o', 'p', 'q', 'r', 's', 't', 'u',
         'v', 'w', 'x', 'y', 'z']

# index array of characters in vocab
v_map = {n: i for i, n in enumerate(vocab)}
v_len = len(v_map)

# training data (character sequences)
# wor -> X, d -> Y
# woo -> X, d -> Y
training_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
test_data = ['wood', 'deep', 'cold', 'load', 'love', 'dear', 'dove', 'cell', 'life', 'keep']

Using TensorFlow backend.


In [0]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []

    for seq in seq_data:
        # Indices of the first three alphabets of the words
        # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
        input = [v_map[n] for n in seq[:-1]]
        # Indices of the last alphabet of the words
        # 3, 3, 15, 4, 3 ...
        target = v_map[seq[-1]]

        # One-hot encoding of the inputs into the sequences of 26-dimensional vectors
        # [0, 1, 2] ==>
        # [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
        input_batch.append(np.eye(v_len)[input])
        
        # We don't apply one-hot encoding for the output,  
        # since we'll use sparse_softmax_cross_entropy_with_logits
        # as our loss function
        target_batch.append(target)

    return input_batch, target_batch


In [0]:
learning_rate = 0.01
n_hidden = 10
total_epoch = 100
n_step = 3 # the length of the input sequence
n_input = n_class = v_len # the size of each input


In [4]:
"""
  Phase 1: Create the computation graph
"""
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# Create an LSTM cell
cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# Create the RNN
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
# output : [batch_size, max_time, cell.output_size]

# Transform the output of RNN to create output values
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [5]:
"""
  Phase 2: Train the model
"""
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    input_batch, target_batch = make_batch(training_data)

    for epoch in range(total_epoch):
        _, loss = sess.run([optimizer, cost],
                           feed_dict={X: input_batch, Y: target_batch})

        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.6f}'.format(loss))

    print('Optimization finished')

    """
      Make predictions
    """
    seq_data = training_data
    prediction = tf.cast(tf.argmax(model, 1), tf.int32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, Y), tf.float32))

    input_batch, target_batch = make_batch(seq_data)

    predict, accuracy_val = sess.run([prediction, accuracy],
                                     feed_dict={X: input_batch, Y: target_batch})

    predicted = []
    for idx, val in enumerate(seq_data):
        last_char = vocab[predict[idx]]
        predicted.append(val[:3] + last_char)

    print('\n=== Predictions ===')
    print('Input:', [w[:3] + ' ' for w in seq_data])
    print('Predicted:', predicted)
    print('Accuracy:', accuracy_val)

Epoch: 0001 cost = 3.372146
Epoch: 0002 cost = 3.301793
Epoch: 0003 cost = 3.231223
Epoch: 0004 cost = 3.159113
Epoch: 0005 cost = 3.084567
Epoch: 0006 cost = 3.006805
Epoch: 0007 cost = 2.925208
Epoch: 0008 cost = 2.839316
Epoch: 0009 cost = 2.748787
Epoch: 0010 cost = 2.653376
Epoch: 0011 cost = 2.552949
Epoch: 0012 cost = 2.447515
Epoch: 0013 cost = 2.337271
Epoch: 0014 cost = 2.222642
Epoch: 0015 cost = 2.104293
Epoch: 0016 cost = 1.983134
Epoch: 0017 cost = 1.860328
Epoch: 0018 cost = 1.737315
Epoch: 0019 cost = 1.615802
Epoch: 0020 cost = 1.497671
Epoch: 0021 cost = 1.384820
Epoch: 0022 cost = 1.278953
Epoch: 0023 cost = 1.181361
Epoch: 0024 cost = 1.092767
Epoch: 0025 cost = 1.013268
Epoch: 0026 cost = 0.942368
Epoch: 0027 cost = 0.879080
Epoch: 0028 cost = 0.822085
Epoch: 0029 cost = 0.769956
Epoch: 0030 cost = 0.721406
Epoch: 0031 cost = 0.675461
Epoch: 0032 cost = 0.631500
Epoch: 0033 cost = 0.589197
Epoch: 0034 cost = 0.548445
Epoch: 0035 cost = 0.509295
Epoch: 0036 cost = 0